# Bright SNe lightcurves

This is based on the Lasair example at https://github.com/lsst-uk/lasair-examples/blob/main/notebooks/API_lsst/BrightSNe.ipynb

In [1]:
import getpass
from lasair import lasair_client as lasair

from panoptes_client import SubjectSet, Panoptes

from src.lsst_zooniverse.generator import LSSTSubjectGenerator, TripletImageLocation

from IPython.display import Image, display


In [2]:
zooniverse_username = input('Zooniverse username')
zooniverse_password = getpass.getpass('Zooniverse password')
API_TOKEN = getpass.getpass('Lasair API token')

In [3]:
Panoptes.connect(username=zooniverse_username, password=zooniverse_password)

In [12]:
PROJECT_ID = 7
SUBJECT_SET_ID = 132831

In [13]:
endpoint = "https://lasair-lsst-dev.lsst.ac.uk/api"

In [14]:
L = lasair(API_TOKEN, endpoint=endpoint)

In [15]:
ra = 51.358273
dec= -27.692442
radii = [100.0, 10, 0]
for radius in radii:
    result = L.cone(ra, dec, radius, requestType='count')
    print('Found %d at radius %f' % (result['count'], radius))

Found 70 at radius 100.000000
Found 0 at radius 10.000000
Found 0 at radius 0.000000


In [16]:
# Just join the objects table and the sherlock_classifications table, just
# pulling out the objectId.
rows = L.query(
    "objects.diaObjectId AS objectId",
    'objects, sherlock_classifications',
    """ sherlock_classifications.classification = "SN"
        AND objects.g_psfFlux > 2000 AND objects.r_psfFlux > 2000
        AND objects.nSourcesGood > 1 
    """,
    limit=10)
objectIds = [row['objectId'] for row in rows]
print(objectIds)

[169342390823813224, 169355605609809259, 169562325818802242, 169562328692424754, 169852561038770458, 313629129740648459, 313637935719645190, 313637935723315254, 313673106899599382, 313673106903269400]


In [17]:
bandColors = {'u': '#9900cc', 'g': '#3366ff', 'r':'#33cc33', 'i':'#ffcc00', 'z':'#ff0000', 'y':'#cc6600'}
glyphs = tuple(('circle', c) for c in bandColors.values())

In [ ]:
subjects = []
for subject_index, subject in enumerate(LSSTSubjectGenerator(objectIds, lasair=L, media_generators=[TripletImageLocation]), start=1):
    subjects.append(subject)
    display(f"Subject {subject_index}")

    for location, media_file in zip(subject.locations, subject._media_files):
        display(Image(data=media_file))

# SubjectSet(SUBJECT_SET_ID).add(subjects)


LasairError: HTTP return code 404 for
https://lasair-lsst-dev.lsst.ac.uk/api/object/